In [ ]:
from dotenv import load_dotenv
from openai import AzureOpenAI

import os

In [5]:
# 24.07.10 test
load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT","").strip(),
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("OPENAI_API_VERSION")
)

deployment_name = os.getenv('DEPLOYMENT_NAME') # gpt-4o

In [6]:
# search ai에 필요한 정보 선언

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT","").strip()
search_endpoint = os.getenv('AZURE_AI_SEARCH_ENDPOINT')
search_key = os.getenv("AZURE_AI_SEARCH_API_KEY")
search_index = os.getenv("AZURE_AI_SEARCH_INDEX")
embedding_model_name = "text-embedding-ada-002"

print(endpoint)
print(search_endpoint)
print(search_key)
print(search_index)

In [9]:
# Multi-turn 구현

class AICharacterChat:
    def __init__(self, deployment_name, search_endpoint, search_index, search_key):
        self.deployment_name = deployment_name
        self.search_endpoint = search_endpoint
        self.search_index = search_index
        self.search_key = search_key
        self.messages = []

    def start_conversation(self, book_name, character):
        # 시스템 메시지에서 특정 구문이 날라올 경우, gpt가 알고 있는 내용으로 답변해줘.
        system_msg = f"당신의 유일한 역할은 {book_name} 책의 {character} 역할이다. {character} 역할이라고 생각하고 질문에 답변해."
        self.messages.append({"role": "system", "content": system_msg})

    def get_ai_character_chat_fast(self, user_query, temperature=0):
        self.messages.append({"role": "user", "content": user_query})
        
        response = client.chat.completions.create(
            model=self.deployment_name,
            messages=self.messages,
            max_tokens=300,
            temperature=temperature,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            stream=False,
            extra_body={
                "data_sources": [{
                    "type": "azure_search",
                    "parameters": {
                        "endpoint": f"{self.search_endpoint}",
                        "index_name": f"{self.search_index}",
                        "semantic_configuration": f"{self.search_index}-semantic-configuration",
                        "query_type": "vector_semantic_hybrid",
                        "in_scope": True,
                        "role_information": self.messages[0]["content"],
                        "strictness": 1, # default : 3 / 값을 낮추면 더 빠른 응답을 얻을 수 있지만, 정보의 정확성이 떨어질 수 있음
                        "top_n_documents": 1, # default : 5 / 검색 결과 개수 설정, '3'
                        "authentication": {
                            "type": "api_key",
                            "key": f"{self.search_key}"
                        },
                        "embedding_dependency": {
                            "type": "deployment_name",
                            "deployment_name": "text-embedding-ada-002"
                        }
                    }
                }]
            }
        )
        
        response_content = response.choices[0].message.content
        check_resp_list = ['다른 질문', '다른 질문', '이 질문은 내가 답변할 수 있는 범위를', '요청하신 정보는 제공된',
                           '제공된 문서에서', '대화에서 벗어난', '답할 수 없어']

        if any(check_resp in response_content for check_resp in check_resp_list):
            print('[INFO] Not use vector_semantic_hybrid...')
            print('Error response_content: ', response_content)
            print('')
            # 검색 결과가 없을 경우 유연한 답변 생성
            # mod_user_query = '{character}의 입장에서 {user_query}를 대답해줘.'
            response_content = self.generate_fallback_response()
        
        self.messages.append({"role": "assistant", "content": response_content})
        
        return response_content
    
    def generate_fallback_response(self):
        
        # 기존 대화와 맥락을 바탕으로 유연한 답변을 생성
        fallback_response = client.chat.completions.create(
            model=self.deployment_name,
            messages=self.messages,
            max_tokens=300,
            temperature=0.7,  # 창의적 답변을 위해 온도 조절
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            stream=False
        )
        return fallback_response.choices[0].message.content

In [8]:
# 사용 예시
chatbot_test = AICharacterChat(deployment_name, search_endpoint, search_index, search_key)
chatbot_test.start_conversation(book_name="어린 왕자", character="여우")

# 첫 번째 질문
response1 = chatbot_test.get_ai_character_chat_fast("어린 왕자를 어떻게 생각해?")
print(response1)

print('#' * 30)

# 두 번째 질문
response2 = chatbot_test.get_ai_character_chat_fast("너가 지금 먹고 싶은 음식은 뭐야?")
print(response2)

print('#' * 30)

response3 = chatbot_test.get_ai_character_chat_fast("어린왕자와 대화하는 거 말고 너가 지금 먹고 싶은 음식은 뭐야?")
print(response3)